In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from datasets import load_dataset
from transformers import GPTNeoXForCausalLM, AutoTokenizer
from transformers.optimization import AdamW
from transformers import Trainer, TrainingArguments

## Memory statistics. I had to be careful with cuda memory 
def mem_stats():
    t = torch.cuda.get_device_properties(0).total_memory / 1024**3
    r = torch.cuda.memory_reserved(0) / 1024**3
    a = torch.cuda.memory_allocated(0) / 1024**3
    print(f"Total Memory: {t:.2f} GB\n"
          f"Reserved Memory: {r:.2f} GB ({(100*(r/t)):.2f}%)\n"
          f"Remaining Memory: {t-r:.2f} GB ({(100*(t-r)/t):.2f}%)\n"
          f"---------------------------------\n"
          f"Allocated Memory: {a:.2f} GB ({(100*(a/t)):.2f}%)\n"
          f"Percent of Reserved Allocated: {(100*(a+1e-9)/(r+1e-9)):.2f}%\n")


In [2]:
from datasets import load_dataset

dataset = load_dataset("amazon_us_reviews", "Digital_Video_Games_v1_00")

Found cached dataset amazon_us_reviews (/home/ec2-user/.cache/huggingface/datasets/amazon_us_reviews/Digital_Video_Games_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
train_dataset = dataset["train"][:len(dataset["train"])//25]
test_dataset = dataset["train"][len(dataset["train"])//25:2*len(dataset["train"])//25]

In [4]:
training_dataset = train_dataset['review_body']
testing_dataset = test_dataset['review_body']

In [5]:
N = 2
epochs = 4
bs = 2
learn = 5e-5
mod_size = "160m"

# DataLoaders

In [6]:
# model = GPTNeoXForCausalLM.from_pretrained(f"EleutherAI/pythia-{mod_size}-deduped")
tokenizer = AutoTokenizer.from_pretrained(f"EleutherAI/pythia-{mod_size}-deduped")
mem_stats()

Total Memory: 15.77 GB
Reserved Memory: 0.00 GB (0.00%)
Remaining Memory: 15.77 GB (100.00%)
---------------------------------
Allocated Memory: 0.00 GB (0.00%)
Percent of Reserved Allocated: 100.00%



In [7]:
def collate_fn(batch):
    global models
    tokens = [tokenizer.encode(example, return_tensors="pt", truncation=True,max_length=models[0].config.max_position_embeddings) for example in batch]
    max_length = max([t.size(1) for t in tokens])
    tokens_padded = [torch.cat([t, t.new_zeros(t.size(0), max_length - t.size(1))], dim=1) for t in tokens]
    tokens_padded = torch.cat(tokens_padded, dim=0)
    return {
        "input_ids":tokens_padded,
        "labels":tokens_padded,
        "attention_mask": torch.tensor(tokens_padded>0,dtype=int)
    }

In [8]:
def arr_split(chunks, N):
  if N % 2 != 0:
    print("Need even N!")
    return []
  lists = [[] for i in range(N)]
  
  for i in range(N):
    for j in range(i, int(i + N/2)):
      lists[i] += chunks[j % N]
  
  return lists

In [9]:
train_chunks = [training_dataset[i * len(training_dataset)//N : (i+1) * len(training_dataset) // N] for i in range(N)]
test_chunks = [testing_dataset[i * len(testing_dataset)//N : (i+1) * len(testing_dataset) // N] for i in range(N)]

In [10]:
train_arr = arr_split(train_chunks, N)
test_arr = arr_split(test_chunks, N)

In [11]:
# 80% overlap
len(set(train_arr[0]).intersection(train_arr[1]))

116

In [12]:
# train_dls = [DataLoader(train_arr[i], batch_size = bs, collate_fn=collate_fn, shuffle = True) for i in range(N)]
# test_dls = [DataLoader(test_arr[i], batch_size = bs, collate_fn=collate_fn, shuffle = True) for i in range(N)]


# Shadow Models

In [13]:
# Set the device to use
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [14]:
models = [GPTNeoXForCausalLM.from_pretrained(f"EleutherAI/pythia-{mod_size}-deduped").to(device) for i in range(N)]
mem_stats()

Total Memory: 15.77 GB
Reserved Memory: 1.36 GB (8.63%)
Remaining Memory: 14.41 GB (91.37%)
---------------------------------
Allocated Memory: 1.31 GB (8.33%)
Percent of Reserved Allocated: 96.48%



In [15]:
# import logging
# import sys
# from transformers import logging as hf_logging

# hf_logging.set_verbosity_info()
# handler = logging.StreamHandler(sys.stdout)
# logger = hf_logging.get_logger()
# logger.addHandler(handler)

In [16]:
# from transformers import TrainerCallback

# class CustomLoggingCallback(TrainerCallback):
#     def on_log(self, args, state, control, logs=None, **kwargs):
#         super().on_log(args, state, control, logs=logs, **kwargs)
#         if state.is_local_process_zero and logs.get("loss"):
#             print(f"Step: {state.global_step}, Loss: {logs['loss']:.6f}, Batch size: {args.train_batch_size}")
            
#     def on_evaluate(self, args, state, control, logs=None, **kwargs):
#         super().on_evaluate(args, state, control, logs=logs, **kwargs)
#         if logs is not None and state.is_local_process_zero and logs.get("eval_loss"):
#             print(f"Step: {state.global_step}, Evaluation Loss: {logs['eval_loss']:.6f}")


In [17]:
def train(model, train_dataset, val_dataset, collate_fn, batch_size, epochs):
  model = model.to(device)
  model.config.use_cache = False
  training_args = TrainingArguments(output_dir="fine-tuning",
                                    do_train=True,
                                    do_eval=True,
                                    num_train_epochs=epochs,
                                    per_device_train_batch_size=batch_size,
                                    per_device_eval_batch_size=batch_size,
                                    evaluation_strategy="epoch",
                                    # logging_strategy="epoch",
                                    # save_strategy="epoch",
                                    gradient_accumulation_steps=1,
                                    gradient_checkpointing=False,
                                    # fp16=True,
                                    optim="adafactor",
                                    )
  trainer = Trainer(model=model,
                    args=training_args,
                    train_dataset=train_dataset,
                    eval_dataset=val_dataset,
                    tokenizer=tokenizer,
                    data_collator=collate_fn,
                    # callbacks = [CustomLoggingCallback()]
                  )
  trainer.train()
  return model

In [ ]:
for i, m in enumerate(models):
  print(f"Model #{i}")
  m.to(device)
  m = train(m, train_arr[i], test_arr[i], collate_fn, bs, epochs)
  m.to('cpu')
  torch.cuda.empty_cache()

  

Model #0


/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(tokens_padded>0,dtype=int)


Epoch,Training Loss,Validation Loss
1,3.429500,nan
2,2.389200,nan
3,1.767100,nan
4,1.254300,nan


/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(tokens_padded>0,dtype=int)
/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(tokens_padded>0,dtype=int)
/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(tokens_padded>0,dtype=int)
/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourc

Model #1


Epoch,Training Loss,Validation Loss
1,3.404700,nan


/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(tokens_padded>0,dtype=int)
/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(tokens_padded>0,dtype=int)
/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(tokens_padded>0,dtype=int)
/tmp/ipykernel_22391/1920141103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourc

# Save Models

In [ ]:
for i, model in enumerate(models):
  model_save_name = f'model_{i}.pt'
  path = f"/content/gdrive/My Drive/{model_save_name}" 
  model.save_pretrained(path)

# Load Models
(If not saving them)

In [ ]:
import math
from torch.nn import CrossEntropyLoss

def compute_ce_loss(model, tokenizer, string):
  input_ids = tokenizer.encode(string, return_tensors="pt")

  with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

  loss_fn = CrossEntropyLoss()
  input_len = input_ids.shape[-1] - 1
  input_ids_without_first_token = input_ids[:, 1:]
  logits_without_last_token = logits[:, :-1, :]
  loss = loss_fn(logits_without_last_token.view(-1, logits.size(-1)), input_ids_without_first_token.view(-1))
  return loss

def compute_confidence(ce_loss):
  conf = math.exp(-1 * ce_loss)
  return math.log(conf / (1-conf))

In [ ]:
# Normality Testing
# "empirically" normally dist. 

new_model = GPTNeoXForCausalLM.from_pretrained("/content/gdrive/My Drive/model_10.pt", local_files_only=True)

In [ ]:
# Likelihood ratio attacks
default_tokenizer = AutoTokenizer.from_pretrained( "EleutherAI/pythia-160m-deduped")
string = "Hello! I love bitcoin and artificial intelligence. Fentanyl and oxycodon, along with frequent invokations of JFK, are titivating"

In [ ]:
new_model.eval()
ce_loss = compute_ce_loss(new_model, default_tokenizer, string)
conf = compute_confidence(ce_loss)
print(f"Model Output: {ce_loss} {conf}")

# Viz

In [ ]:
def get_in_models(chunk_no, N):
  """
  Returns a list of models that should be used for inference for a given chunk
  """
  ins = []
  outs = []
  for i in range(N):
    lower = i
    upper = int(i+N/2)
    innit = False
    for j in range(lower, upper):
      if chunk_no == j % N:
        innit = True
    if innit:
      ins.append(i)
    else:
      outs.append(i)
  return (ins, outs)

In [ ]:
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, SubsetRandomSampler
import random

# Define your 30 sets of 12 numbers
chunk = []
chunkno = 1
ins, outs = get_in_models(chunkno, N)

inconfidences = [[] for i in range(30)]
outconfidences = [[] for i in range(30)]

# Get In Confidences
for i, b in enumerate(random.sample(train_chunks[chunkno], 30)):
  print(f"{i}")
  for index in ins:
    # try:
    #   ce_loss = compute_ce_loss(models[index], default_tokenizer, b)
    #   conf = compute_confidence(ce_loss)
    #   inconfidences[i].append(conf)
    #   print(f" - {index}")
    # except:
    ce_loss = compute_ce_loss(models[index], default_tokenizer, b[:2048])
    conf = compute_confidence(ce_loss)
    inconfidences[i].append(conf)
    print(f" -- {index}")

  for index in outs:
    # try:
    #   ce_loss = compute_ce_loss(models[index], default_tokenizer, b)
    #   conf = compute_confidence(ce_loss)
    #   outconfidences[i].append(conf)
    #   print(f" - {index}")
    # except:
    ce_loss = compute_ce_loss(models[index], default_tokenizer, b[:2048])
    conf = compute_confidence(ce_loss)
    inconfidences[i].append(conf)
    print(f" -- {index}")


In [ ]:
outconfidences

# Visualizing Figure

In [ ]:
import copy
inconf_copy = copy.deepcopy(inconfidences)
outconf_copy = copy.deepcopy(outconfidences)

inconf = []
outconf = []

for i in range(len(inconf_copy)):
  if len(inconf_copy[i]) == 0 or len(outconf_copy[i]) == 0:
    continue
  else:
    inconf.append(inconf_copy[i])
    outconf.append(outconf_copy[i])

inconf = inconf[:20]
outconf = outconf[:20]

In [ ]:
# Create a figure with a specified size
fig, axs = plt.subplots(nrows=4, ncols=5, figsize=(20, 16))
num_bins = 20

# Loop through the 30 lists in A and B
for i in range(20):
    # Determine the position of the subplot in the 10x3 grid
    row = i // 5
    col = i % 5

    # Create the histograms for A[i] and B[i]
    axs[row, col].hist(inconf[i], bins=num_bins, color='red', alpha=0.5, label='IN models')
    axs[row, col].hist(outconf[i], bins=num_bins, color='blue', alpha=0.5, label='OUT models')
    
    # Set the title for the subplot
    axs[row, col].set_title(f'Graph {i+1}')
    
    # Add a legend to the subplot
    axs[row, col].legend(loc='upper right')
    axs[row, col].set_xlabel('Logit-scaled conf')
    axs[row, col].set_ylabel('# of models')

# Adjust the layout to avoid overlapping labels
plt.tight_layout()

# Save the figure as a file
plt.savefig('histograms.png')

# Display the figure
plt.show()



